In [1]:
import pandas as pd
import numpy as np
import sklearn as sklearn
%matplotlib inline

# Exercise 3:

A credit card company is ramping up their Data Science Department and just hired you. They have just integrated their data on credit card usage and personal information (like education and location). Further, since they have a personal loans department, they also know, for the people that took a loan, their annual income, credit score, and whether they own a home.

They have 5.2 million clients, but only 100k of them* have taken a loan. Below is the data for each of the 100k, along with their income.

Your goal is to create a model that predicts a person's annual income given all the other features.

* for simplicity, assume that those 100k properly represent the 5.2M and don't have any sampling biases

In [2]:
income_matrix = pd.read_csv('income_matrix_ex3.csv', compression='gzip')
income_matrix.head()

,number_of_degrees,has_technical_degree,max_spend_in_a_month,num_purchases_last_3_months,credit_score,owns_home,lived_in_NYC,lived_in_SF,income
0,1.0,False,4178.724200,456,714.0,False,False,False,88.448668
1,0.0,False,3501.739244,86,669.0,True,False,False,39.494271
2,1.0,False,3298.700962,39,680.0,True,False,False,84.836224
3,0.0,False,3052.162814,31,697.0,True,False,True,132.722244
4,1.0,True,2601.803821,141,702.0,False,False,True,77.237528


In [3]:
income_matrix["lived_in_SF"] = np.where(income_matrix["lived_in_SF"], 1, 0)
income_matrix["lived_in_NYC"] = np.where(income_matrix["lived_in_NYC"], 1, 0)
income_matrix["owns_home"] = np.where(income_matrix["owns_home"], 1, 0)
income_matrix["has_technical_degree"] = np.where(income_matrix["has_technical_degree"], 1, 0)

In [4]:
y = income_matrix["income"]
X = income_matrix.drop(["income"], axis = 1)

## 3A1

Train a model with 20% on the test set.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
from sklearn.preprocessing import MinMaxScaler
norm_features = ['number_of_degrees', 'max_spend_in_a_month', 'credit_score']
mm_scaler = MinMaxScaler()

mm_scaler.fit(X_train[norm_features])
X_train[norm_features] = mm_scaler.transform(X_train[norm_features])
X_test[norm_features] = mm_scaler.transform(X_test[norm_features ])

/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [7]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

## 3A2

How would you describe the accuracy of your model?

In [8]:
reg.score(X_test, y_test)

0.22006966581774134

In [9]:
from sklearn.metrics import explained_variance_score
y_pred = reg.predict(X_test)
explained_variance_score(y_test, y_pred)

0.22008399444758053

I would use the explained variance (adjusted R score) over the R score because it adjusts the score for the number of features used (this can lead to overfitting) whereas the r score does not make this adjustment. The R score would simply increase by default the more features we add, which is not what we want. However in our case we can see that they are very similar, which is an indication of less overfitting. Furthermore, as can be seen above, there is only a 22.6% accuracy for the model. This means that the model is not a very good predictor.

## 3B

Your company has just struck a partnership with Tesla! Part of this deal means that you now have to identify the top earners in your database (to send them ads). For each person that buys a car through your ad, your company will get 0.85% of the value of the purchase.

The CMO has taught you that it is well-known that one's income (in particular making over 165k) is all that matters when it comes to the likelihood of buying a Tesla, and he has decided to allocate budget to advertise to 260k people. Now 0.85% of a Tesla is a lot, so you better pick the right people! Describe how your model will do in picking the right 260k people. Please write down a sentence or two explaining why you picked that metric

This suddenly becomes a classification problem where the top x% by salary are 1 and the others 0. To pick the right 260k people I will find the proportion 260k people is out of 5.2 million and then knowing this I will take the top x% by true salary and classify them as 1 and the rest as 0. Then using the predicted incomes I will do the same. I would be optimizing the precision with this approach because 0.85% of a Tesla is a lot which means that the cost of a false positive is very high. Therefore, the precision is a good metric because it minimizes the false positives. 

In [10]:
p_ads = 260000/5200000

ad_qty = int(y_test.shape[0]*p_ads)
ad_cutoff = np.partition(y_test, -ad_qty)[-ad_qty]

In [11]:
y_test_ad = np.where(y_test>=ad_cutoff, 1, 0)
y_test_ad

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
ad_qty = int(y_pred.shape[0]*p_ads)
ad_cutoff = np.partition(y_pred, -ad_qty)[-ad_qty]
y_pred_ad = np.where(y_pred>=ad_cutoff, 1, 0)

In [13]:
from sklearn.metrics import precision_score
ad_precision = precision_score(y_test_ad, y_pred_ad, average='macro') 
print("{} of all the ads will be properly targeted".format(ad_precision))

0.6247368421052631 of all the ads will be properly targeted


## 3C

The Tesla campaign has gone so fantastically well that your CEO has decided to do something good for the community. Yay! She has created a pro bono partnership with the local hospitals to help low income residents understand their options for treatment. You will help them identify all residents that you think earn less than the median annual income (which are the ones eligible for benefits).

How does your model do against that goal? Please write down a sentence or two explaining why you picked that metric.

In [21]:
median_income = y_train.median()

In [23]:
y_test_ben = np.where(y_test<median_income, 1, 0)
y_pred_ben = np.where(y_pred<median_income, 1, 0)

Here I would use an F1 score because I think that both the false positive and false negatives have high costs. The false positives are bad for the company because they are handing out benefits to people that have a higher than median salary, so the money would be wasted on people that do not need the free money. On the other hand false negatives are families that need the benefits but do not receive them which is also very bad.

In [29]:
from sklearn.metrics import f1_score
my_f1_score = f1_score(y_test_ben, y_pred_ben, average='macro')  
print("F1 score: {}".format(my_f1_score))

F1 score: 0.6746347823443146


## 3D

Your company is doing so well that almost everyone in the US has a credit card with you! Continuing with the pro bono work, a few small cities (of around 200-400 people each) have decided that they want to start collecting city taxes, and your company has offered to help them estimate how much taxes they could raise.

All these cities will charge about 2.5% city-tax, and your goal is to estimate as precisely as possible how much tax revenue will be brought. How do you expect your model to do against that goal?  Please write down a sentence or two explaining why you picked that metric.

Here is all about mazimizing the accuracy of the model and in our case to avoid overfitting, we would need to maximize the r score adjusted. I do not expect my model to do very well against this goal because it has a very low r score adjusted of 22% for predicting income. I cannot isolate how it would do for small cities as I do not have the appropriate features for this. Using the general model's results is enough eitherway because it represents all american populations reasonably well.